<a href="https://colab.research.google.com/github/luthandonx/Python/blob/main/DNA_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Luthando Nxumalo

In [ ]:
#Imports
import pandas as pd
import numpy as np
import re
from numpy.lib import type_check
import numpy as np
import tensorflow as tf
import keras
from keras import utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, SimpleRNN
from sklearn.model_selection import train_test_split

In [ ]:
#Reading the Data
train_data = pd.read_csv("/content/training_data.csv")
#Combining the Train sequences

Train_PRSeq = train_data["PR Seq"]
Train_RTSeq = train_data["RT Seq"]
Train_Y = train_data["Resp"]
np_PRSeq = np.array(Train_PRSeq)
np_RTSeq = np.array(Train_RTSeq)
Train_X = []
for x in range(len(Train_RTSeq)):
  word = Train_RTSeq[x]+""+Train_RTSeq[x]
  Train_X.append(word)
Train_X = np.array(Train_X)


#Variance in Length of data
length_values = []

#finding shortest and longest sequences
longest = len(Train_X[0])
shortest = longest
for l in Train_X:
  length_values.append(len(l))
  if (longest <= len(l)):
    longest = len(l)
  if (shortest >= len(l)):
    shortest = len(l)
print("Longest: ",longest)
print("shortest: ",shortest)

#calculate mean
mean = sum(length_values)/float(len(length_values))
print("Mean: ",mean)

#Calculate varience 
varience = sum((i - mean)**2 for i in length_values)/float(len(length_values))
print("Varience: ", (varience)**(1/2))
#Encode the sequences

def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def string_to_array(seq_string):
  seq_string = seq_string.lower()
  seq_string = re.sub('[^acgt]', 'n', seq_string)
  seq_string = np.array(list(seq_string))
  return seq_string
# create a label encoder with 'acgtn' alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z','n']))
def Label_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 2 # A
    float_encoded[float_encoded == 1] = 5 # C
    float_encoded[float_encoded == 2] = 7 # G
    float_encoded[float_encoded == 3] = 0 # Anything else
    float_encoded[float_encoded == 4] = 1 # T
    return float_encoded


#Encode the entire sequences
train_x = []
count = -1
test  = []
for x in Train_X:
  train_x.append(Label_encoder(string_to_array(x)))
train_x = tf.keras.preprocessing.sequence.pad_sequences( train_x, padding="post")
pretrain= np.array(train_x)
train_x = np.array(train_x)
np_train_y = np.array(Train_Y)

def Kmers_funct(seq, size):
    return [seq[x:x+size] for x in range(len(seq) - size + 1)]
name = []
frame = []
fin = []
c = 0
for x in train_x:
  name =Kmers_funct(x, size=4)
  stri = ""
  for n in name:
    stri = ''.join(map(str, n))
    frame.append(stri)
  fin.append( ' '.join(map(str, frame)))
  frame = []


batch_size = 5
token = Tokenizer()
token.fit_on_texts(fin)
word_index = token.word_index
train_x = pad_sequences(token.texts_to_sequences(fin),maxlen = 2961)

#Masking the Mask 0
for x in range(len(train_x)):
  for y in range(len(train_x[x])):
    if train_x[x][y] == 1:
      train_x[x][y]=0
  
x_train,x_test,y_train,y_test = train_test_split(train_x , np_train_y,test_size=0.1,random_state =5)

FileNotFoundError: ignored

The model should not be sensitive to the padding therefore we should use a mask

In [ ]:
model = Sequential()
model.add(Embedding(1024, 64, mask_zero=True))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.4))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid', input_shape=(None, 2000)))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta = 0.0005, patience=8, restore_best_weights=True )
history = model.fit(x_train, y_train, batch_size=5 , epochs=200, validation_data=(x_test, y_test))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          65536     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 50)                6450      
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 138,085
Trainable params: 138,085
Non-trainable params: 0
__________________________________________________

In [ ]:
model = Sequential()
model.add(Embedding(500 , 64 , mask_zero=True))
model.add(Bidirectional(LSTM(64 , activation='ReLU' )))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

history=model.fit(x_train, y_train,
                  batch_size=100,
                  epochs=10,
                  validation_data=[x_test, y_test])


print(history.history['loss'])
print(history.history['accuracy'])

LSTM layer should be in 3D shape i.e. (samples, time-steps, features). 

The samples are the number of samples in the input data

The time-steps is the number of time-steps per sample

features correspond to the number of features per time-step

LSTM layer has parameter return_sequences, which is set to True. When the return sequence is set to True, the output of the hidden state of each neuron is used as an input to the next LSTM layer

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

AttributeError: ignored